### KNN Classification(Discrete)

In [1]:
import pandas as pd
import numpy as np

- Read Training Data

In [2]:
Training_df = pd.read_csv('KNN_Discrete.csv')
Training_df

,F1,F2,F3,F4,Class
0,1.0,0.5,0.44,0.00,l
1,0.8,0.6,0.00,0.22,l
2,0.7,0.4,0.00,0.00,l
3,0.0,0.1,2.64,4.40,c
4,0.1,0.0,1.76,12.32,c


- Split Features and Target

In [3]:
Target = Training_df['Class']
Features = Training_df.drop(columns = 'Class')
Target,Features

(0    l
 1    l
 2    l
 3    c
 4    c
 Name: Class, dtype: object,
     F1   F2    F3     F4
 0  1.0  0.5  0.44   0.00
 1  0.8  0.6  0.00   0.22
 2  0.7  0.4  0.00   0.00
 3  0.0  0.1  2.64   4.40
 4  0.1  0.0  1.76  12.32)

In [4]:
Test_df = pd.read_csv('KNN_Real_query.csv')
Test_df

,F1,F2,F3,F4
0,0.5,0.6,0.22,0.00
1,0.1,0.0,2.64,0.88


#### Class implementation

In [62]:
class KNNDiscrete:
    def fit(self, Features: pd.DataFrame, Target: pd.Series,K: int):
        self.Features = Features
        self.Target = Target
        self.K = K
        
    def eucDist(self,_feature: pd.Series,_test: pd.Series):
        euc_dist = np.sqrt(np.sum(np.square(_feature-_test)))
        return euc_dist
    
    def predictDiscrete(self,Test):
        self.target_arr = []
        self.Test = Test
        for i in range(len(Test)):
            self.euc_dist_dict = {}
            for j in range(len(Features)):
                self.euc_dist_dict[self.eucDist(self.Features.iloc[j],self.Test.iloc[i])] = self.Target[j]
                
            self.target_arr.append(self.getActual(self.euc_dist_dict))
            
        self.Test['Predicted'] = self.target_arr
        return self.Test
    
    def predict_weight(self,Test):
        self.Test = Test
        self.target_arr = []
        for i in range(len(Test)):
            self.euc_dist_dict = {}
            for j in range(len(Features)):
                    self.euc_dist_dict[self.eucDist(self.Features.iloc[j],self.Test.iloc[i])] = self.Target[j]
                
            self.target_arr.append(self.get_Actual_Weight_Class(self.euc_dist_dict))
            
        Test['Predicted'] = self.target_arr
        return Test
        
    
    def eucWeight(self,dist: float):
        return (1/np.square(dist))

    def distance_To_Weight(self,euc_dist_dict: dict):
        temp_dict = {}
        for key,value in euc_dist_dict.items():
            temp_dict[self.eucWeight(key)] = value

        return temp_dict
        
    def get_Actual_Weight_Class(self,euc_dist_dict: dict):
        _class_weight_dict = {}
        #Slice first k elements
        euc_dist_dict = self.slice_k(euc_dist_dict)
        #Convert key from (dist to weight)
        euc_weight_dict = self.distance_To_Weight(euc_dist_dict)
        #Sum up the weights for the respective labels
        for value in euc_weight_dict.values():
            _class_weight_dict[value] = np.sum([k for k,items in euc_weight_dict.items() if value == items])

        return list(_class_weight_dict.keys())[list(_class_weight_dict.values()).index(max(_class_weight_dict.values()))]
    
    def getActual(self,euc_dist_dict: dict):
        _class_count_dict = {}
        euc_dist_dict = self.slice_k(euc_dist_dict)
        
        for value in euc_dist_dict.values():
            _class_count_dict[value] = _class_count_dict.get(value,0)+1
        
        key = list(_class_count_dict.keys())[list(_class_count_dict.values()).index(max(_class_count_dict.values()))]
        
        return key
        
    def slice_k(self,euc_dist_dict: dict):
        #Sorting dictionary with key
        euc_dist_dict = dict(sorted(euc_dist_dict.items()))
        #Converting to list and slicing and then converting to dict
        return dict(list(euc_dist_dict.items())[:self.K])

#### - Fitting Model

In [63]:
model = KNNDiscrete()
model.fit(Features,Target,3)

#### - Dist Prediction

In [65]:
predicted_df = model.predictDiscrete(Test_df)
predicted_df

,F1,F2,F3,F4,Predicted
0,0.5,0.6,0.22,0.00,l
1,0.1,0.0,2.64,0.88,l


#### - Weight Prediction

In [64]:
model.predict_weight(Test_df)

,F1,F2,F3,F4,Predicted
0,0.5,0.6,0.22,0.00,l
1,0.1,0.0,2.64,0.88,l
